# Clean USIS Dataset
**Author:** Jacob Kvasnicka <br>
**Date:** September 24, 2024

## Setup

In [1]:
import pandas as pd
import numpy as np
import random
import json

from config_management import UnifiedConfiguration
from raw_processing.cehd_cleaning import CehdCleaner
from raw_processing import cehd_loading
from raw_processing import usis
from raw_processing.tests import test_cehd, test_usis

config = UnifiedConfiguration()

In [2]:
expected_cehd = test_cehd.load_test_data(config.path)

In [2]:
raw_cehd = cehd_loading.raw_chem_exposure_health_data(config.cehd, config.path)

cehd_cleaner = CehdCleaner(config.path, config.cehd)

cehd_data = cehd_cleaner.clean_raw_data(raw_cehd)

Loading 1984 data...
Loading 1985 data...
Loading 1986 data...
Loading 1987 data...
Loading 1988 data...
Loading 1989 data...
Loading 1990 data...
Loading 1991 data...
Loading 1992 data...
Loading 1993 data...
Loading 1994 data...
Loading 1995 data...
Loading 1996 data...
Loading 1997 data...
Loading 1998 data...
Loading 1999 data...
Loading 2000 data...
Loading 2001 data...
Loading 2002 data...
Loading 2003 data...
Loading 2004 data...
Loading 2005 data...
Loading 2006 data...
Loading 2007 data...
Loading 2008 data...
Loading 2009 data...
Loading 2010 data...
Loading 2011 data...
Loading 2012 data...
Loading 2013 data...
Loading 2014 data...
Loading 2015 data...
Loading 2016 data...
Loading 2017 data...
Loading 2018 data...
Loading 2019 data...
Loading 2020 data...
Loading 2021 data...
Loading 2022 data...
Loading 2023 data...


C:\Users\JKVASNIC\AppData\Local\miniconda3\envs\chiu-lab\Lib\site-packages\pandas\core\dtypes\astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)


In [5]:
expected_usis = test_usis.load_test_data(config.path)

In [3]:
raw_usis = usis.load_raw_usis_data(config.path['raw_usis_file'])

# usis_cleaner = usis.UsisCleaner(config.path, config.usis)

# usis_data = usis_cleaner.clean_raw_data(raw_usis)

In [25]:
pd.to_numeric(expected_cehd['sampling_number'], errors='coerce').astype('flat')

TypeError: data type 'flat' not understood

In [51]:
# Be mindful that some CEHD samples already indicate 'TWA'
groupby_columns = [
    'inspection_number',
    'imis_substance_code'
]

expected_cehd.loc[expected_cehd['eight_hour_twa_calc']=='Y'].groupby(groupby_columns)['sampling_number'].value_counts().sort_values(ascending=False)

inspection_number  imis_substance_code  sampling_number
1399877            H130                 346115             20
                   1377                 346115             20
                   1539                 346115             20
302400478          1371                 400565339          19
                   0810                 400565339          19
                                                           ..
303324958          9135                 400077707           1
                                        400077681           1
                   2610                 400077699           1
                   2571                 400077699           1
999784             C141                 191395              1
Name: sampling_number, Length: 43690, dtype: int64

In [22]:
# A single sampling number can have several field numbers with different results
mask = (
    (expected_cehd['inspection_number']=='1399877') 
    & (expected_cehd['imis_substance_code']=='H130')
    & (expected_cehd['sampling_number']=='346115')
)
expected_cehd.loc[mask, 'time_sampled']

index
2488194    15.0
2488197    13.0
2488200    13.0
2488203    12.0
2488206    14.0
2488209    15.0
2488212    14.0
2488215    13.0
2488218    12.0
2488221    14.0
2488224    12.0
2488227    14.0
2488230    14.0
2488233    14.0
2488236    14.0
2488239    14.0
2488242    14.0
2488245    13.0
2488248    10.0
2488251    12.0
Name: time_sampled, dtype: float64

In [40]:
# USIS has full-shift TWA exposure type
groupby_columns = [
    'inspection_number',
    'substance_id'
]

expected_usis.loc[expected_usis['exposure_type_id']=='T'].groupby(groupby_columns)['sheet_number'].value_counts()

inspection_number  substance_id  sheet_number
1749               0040          559             1
                   0372          558             1
                   0645          552             1
                   1377          1033            1
                   1650          552             1
                                                ..
318001260          9130          436809446       1
318002185          9010          436809628       1
                                 436809636       1
318003324          9130          436809784       1
                   9135          436809792       1
Name: sheet_number, Length: 433225, dtype: int64

In [27]:
# USIS sheet number appears to be the CEHD sampling number
intersection = set(expected_cehd['sampling_number']).intersection(expected_usis['sheet_number'].astype('str'))

len(intersection)

40577

In [ ]:
# Decide how to aggregate samples

In [ ]:
# Maybe use the smaller of the two datasets as external validation set

In [ ]:
# Use the same column names for shared variables like sampling number

In [ ]:
# Get DTXSIDs for substance IDs
# Map NAICS codes to names

In [30]:
f = r"C:\Users\JKVASNIC\Repositories\ht-occupational-plus\input\raw\osha\cehd_usis_substance_union.txt"
with open(f, 'r', encoding='utf-8') as file:
    print(len(file.readlines()))

1453


In [50]:
f = r"C:\Users\JKVASNIC\Repositories\ht-occupational-plus\input\raw\osha\CCD-Batch-Search_2024-09-11_11_44_23.csv"
chem_id_for_name = pd.read_csv(f).set_index('INPUT')['DTXSID'].dropna().to_dict()

In [55]:
[chem_id_for_name.get(name, np.nan) for name in expected_cehd['substance']]

[nan,
 nan,
 nan,
 'DTXSID8021434',
 'DTXSID2021284',
 'DTXSID2025050',
 'DTXSID2025050',
 'DTXSID2025050',
 'DTXSID2025050',
 'DTXSID2025050',
 'DTXSID6022422',
 'DTXSID6022422',
 'DTXSID6022422',
 'DTXSID6022422',
 'DTXSID6022422',
 'DTXSID2025050',
 'DTXSID6022422',
 'DTXSID6022422',
 'DTXSID2025050',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 'DTXSID2025553',
 'DTXSID1024097',
 'DTXSID1024097',
 'DTXSID1024097',
 'DTXSID1024097',
 'DTXSID1024097',
 'DTXSID1024097',
 'DTXSID1024097',
 'DTXSID1024097',
 'DTXSID3021516',
 'DTXSID3021516',
 'DTXSID3021516',
 'DTXSID3021516',
 'DTXSID3021516',
 'DTXSID8021482',
 'DTXSID8021482',
 'DTXSID8021482',
 'DTXSID8021482',
 'DTXSID8021482',
 'DTXSID8021482',
 'DTXSID8021482',
 'DTXSID5021889',
 'DTXSID5021889',
 'DTXSID5021889',
 'DTXSID7021360',
 'DTXSID7021360',
 'DTXSID7021360',
 'DTXSID7021360',
 'DTXSID7021360',
 'DTXSID7021360',
 'DTXSID7021360',
 'DTXSID7021360',
 'DTXSID7021360',
 'DTXSID7021360',
 'DTXSID7021360',
 'DTXSID8020759',
 'DTXSID80

In [8]:
exposure_data = expected_cehd.copy()

In [20]:
exposure_data.groupby('inspection_number')['imis_substance_code'].nunique().sort_values(ascending=False)

inspection_number
944576       35
105939714    33
1841055      33
113462048    32
101450294    32
             ..
116134628     1
305125189     1
116135948     1
116136508     1
019-17        1
Name: imis_substance_code, Length: 59480, dtype: int64

In [19]:
# A single sampling number can have several field numbers with different results
mask = (
    (expected_cehd['inspection_number']=='106740848') 
    & (expected_cehd['imis_substance_code']=='9020')
)
expected_cehd.loc[mask]

,air_volume_sampled,blank_used,city,date_reported,date_sampled,eight_hour_twa_calc,establishment_name,field_number,imis_substance_code,inspection_number,...,state,substance,time_sampled,year,zip_code,censored,unit_of_measurement,sample_weight,sample_result,instrument_type
index,,,,,,,,,,,,,,,,,,,,,
664377,61.5,N,TRENTON,1989-05-03,1989-03-02,NaN,"FRICTION DIVISION PRODUCTS, INC.",914-421,9020,106740848,...,NJ,Asbestos (all forms),41.0,1989,08658,N,F,0.0,0.50,ID160
664378,82.5,N,TRENTON,1989-05-03,1989-03-02,NaN,"FRICTION DIVISION PRODUCTS, INC.",914-474,9020,106740848,...,NJ,Asbestos (all forms),55.0,1989,08658,N,F,0.0,0.00,ID160
664379,66.0,N,TRENTON,1989-05-03,1989-03-02,NaN,"FRICTION DIVISION PRODUCTS, INC.",914-498,9020,106740848,...,NJ,Asbestos (all forms),44.0,1989,08658,N,F,0.0,0.00,ID160
664387,58.5,N,TRENTON,1989-05-03,1989-03-02,NaN,"FRICTION DIVISION PRODUCTS, INC.",914-423,9020,106740848,...,NJ,Asbestos (all forms),39.0,1989,08658,N,F,0.0,1.87,ID160
664388,42.0,N,TRENTON,1989-05-03,1989-03-02,NaN,"FRICTION DIVISION PRODUCTS, INC.",914-432,9020,106740848,...,NJ,Asbestos (all forms),28.0,1989,08658,N,F,0.0,1.67,ID160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664907,100.5,N,TRENTON,1989-03-29,1989-02-15,NaN,"FRICTION DIVISION PRODUCTS, INC.",914-262,9020,106740848,...,NJ,Asbestos (all forms),67.0,1989,08658,N,F,0.0,0.55,ID160
664908,36.0,N,TRENTON,1989-03-29,1989-02-15,NaN,"FRICTION DIVISION PRODUCTS, INC.",914-269,9020,106740848,...,NJ,Asbestos (all forms),24.0,1989,08658,N,F,0.0,2.25,ID160
664909,52.5,N,TRENTON,1989-03-29,1989-02-15,NaN,"FRICTION DIVISION PRODUCTS, INC.",914-281,9020,106740848,...,NJ,Asbestos (all forms),35.0,1989,08658,N,F,0.0,0.19,ID160


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Assuming 'exposure_data' is your initial DataFrame containing the data
# Adjust the column names in 'columns' dictionary to match your DataFrame

# Define the column names used in your DataFrame
columns = {
    'inspection_number': 'inspection_number',    # Inspection Number
    'substance_code': 'substance_code',          # Substance Code (Chemical Identifier)
    'sampling_number': 'sampling_number',        # Sampling Number
    'field_number': 'field_number',              # Field Number
    'concentration': 'concentration',            # Concentration Measurement
    'sampling_time': 'sampling_time',            # Sampling Time in minutes
    'NAICS_code': 'NAICS_code',                  # NAICS Sector Code
    # Add or adjust column names as necessary
}

# Step 1: Compute the median concentration across field numbers for each combination of inspection_number, substance_code, and sampling_number
median_concentration = (
    exposure_data
    .groupby([
        columns['inspection_number'],
        columns['substance_code'],
        columns['sampling_number']
    ])[columns['concentration']]
    .median()
    .reset_index()
    .rename(columns={columns['concentration']: 'median_concentration'})
)

# Merge the sampling time into median_concentration
sampling_times = (
    exposure_data
    .groupby([
        columns['inspection_number'],
        columns['substance_code'],
        columns['sampling_number']
    ])[columns['sampling_time']]
    .mean()
    .reset_index()
)

# Merge median_concentration with sampling_times
median_concentration = median_concentration.merge(
    sampling_times,
    on=[
        columns['inspection_number'],
        columns['substance_code'],
        columns['sampling_number']
    ],
    how='left'
)

# Step 2: Calculate the TWA for each inspection_number and substance_code
# Calculate Ci * ti
median_concentration['Ci_ti'] = (
    median_concentration['median_concentration'] *
    median_concentration[columns['sampling_time']]
)

# Sum Ci_ti and sampling_time for each inspection_number and substance_code
TWA_per_inspection = (
    median_concentration
    .groupby([
        columns['inspection_number'],
        columns['substance_code']
    ])
    .agg({
        'Ci_ti': 'sum',
        columns['sampling_time']: 'sum'
    })
    .reset_index()
)

# Calculate TWA = sum(Ci * ti) / sum(ti)
TWA_per_inspection['TWA'] = (
    TWA_per_inspection['Ci_ti'] /
    TWA_per_inspection[columns['sampling_time']]
)

# Step 3: Merge NAICS_code into TWA_per_inspection
# Get the first NAICS_code for each inspection_number
NAICS_per_inspection = (
    exposure_data
    .groupby(columns['inspection_number'])[columns['NAICS_code']]
    .first()
    .reset_index()
)

# Merge NAICS_code into TWA_per_inspection
TWA_per_inspection = TWA_per_inspection.merge(
    NAICS_per_inspection,
    on=columns['inspection_number'],
    how='left'
)

# Step 4: Aggregate TWA values across inspections for each substance_code and NAICS_code
# Remove any non-positive TWA values before computing the geometric mean
positive_TWA = TWA_per_inspection[TWA_per_inspection['TWA'] > 0].copy()

# Define a function to compute the geometric mean
def geometric_mean(x):
    return np.exp(np.mean(np.log(x)))

# Group by substance_code and NAICS_code and compute the geometric mean and count
aggregated_TWA = (
    positive_TWA
    .groupby([columns['substance_code'], columns['NAICS_code']])
    .agg(
        TWA_geomean=('TWA', geometric_mean),
        sample_count=('TWA', 'count')
    )
    .reset_index()
)

# Step 5: Calculate EC for each chemical-sector combination using EC ≈ TWA * 0.2283
aggregated_TWA['EC'] = aggregated_TWA['TWA_geomean'] * 0.2283

# Step 6: One-hot encode NAICS_code
NAICS_dummies = pd.get_dummies(aggregated_TWA[columns['NAICS_code']], prefix='NAICS')

# Combine one-hot encoded NAICS codes with aggregated_TWA
aggregated_TWA = pd.concat([aggregated_TWA, NAICS_dummies], axis=1)

# Step 7: Merge chemical descriptors into aggregated_TWA
# Assuming you have a DataFrame 'chemical_descriptors' with 'substance_code' and descriptor columns
# Adjust the column names and path as necessary

# Example:
# chemical_descriptors = pd.read_csv('chemical_descriptors.csv')  # Load your chemical descriptors data

# Merge chemical_descriptors into aggregated_TWA on 'substance_code'
# aggregated_TWA = aggregated_TWA.merge(
#     chemical_descriptors,
#     on=columns['substance_code'],
#     how='left'
# )

# Ensure that 'chemical_descriptors' DataFrame exists and contains the correct columns
# For the purpose of this code, we'll assume 'chemical_descriptors' is already defined

# Step 8: Prepare the final dataset with features and target variable
# The features are chemical descriptors and one-hot encoded NAICS codes
# The target variable is 'EC'

# List of feature columns (excluding 'substance_code' and 'NAICS_code')
feature_columns = list(NAICS_dummies.columns)  # Start with one-hot encoded NAICS codes

# If chemical descriptors are included, add their column names to 'feature_columns'
# chemical_descriptor_columns = [col for col in chemical_descriptors.columns if col != 'substance_code']
# feature_columns.extend(chemical_descriptor_columns)

# Ensure there are no missing values in the features or target variable
# aggregated_TWA.dropna(subset=feature_columns + ['EC'], inplace=True)

# The final dataset is 'aggregated_TWA' with features in 'feature_columns' and target variable 'EC'

# Output the final prepared data (optional)
# aggregated_TWA.to_csv('prepared_data.csv', index=False)

# The data is now ready for modeling
# Features: aggregated_TWA[feature_columns]
# Target: aggregated_TWA['EC']